---
toc: False
comments: true
title: PPR
layout: post 
type: hacks
description: Personal Project Requierments
courses: { compsci: {week: 23} }
---

1. function
2. call to function
3. list
4. list in use

## Function 

In [ ]:
@search_api.route("/increment_tag", methods=["POST"])
@token_required()
def increment_tag():
    data = request.get_json()
    if not data or "name" not in data:
        return jsonify({"error": "Invalid data"}), 400

    current_user = g.current_user  
    item_name = data["name"]

    item = next((item for item in items if item["name"].lower() == item_name.lower()), None)
    if item:

        for tag in item["tags"]:
            item["tags"][tag] += 1

        try:
            search_entry = SearchHistory(
                user=current_user.uid,
                name=item["name"],  
                tags=item["tags"],  
                query=None 
            )
            db.session.add(search_entry)
            db.session.commit()

            append_to_json(search_entry.read())
        except IntegrityError as e:
            db.session.rollback()
            return jsonify({"error": f"Failed to log item selection: {str(e)}"}), 500

        return jsonify({
            "message": f"Tags for '{item_name}' updated successfully!",
            "tags": item["tags"]
        }), 200

    return jsonify({"error": "Item not found!"}), 404

## Call To Function

In [ ]:
async function searchItems() {
    const input = document.getElementById('searchInput').value.trim().toLowerCase();
    const resultsDiv = document.getElementById('results');
    resultsDiv.innerHTML = ''; // Clear previous results

    if (input) {
      try {
        const response = await fetch(`${pythonURI}/api/search?q=${encodeURIComponent(input)}`, {
          ...fetchOptions,
          method: 'GET',
          headers: { 'Content-Type': 'application/json' }
        });

        if (!response.ok) {
          throw new Error(`HTTP error! Status: ${response.status}`);
        }

        const items = await response.json();
        if (items.length > 0) {
          items.forEach(item => {
            const resultDiv = document.createElement('div');
            resultDiv.className = 'result';
            resultDiv.textContent = item.name;
            resultDiv.onclick = async () => {
              await incrementTags(item.name);
              window.location.href = item.link;
            };
            resultsDiv.appendChild(resultDiv);
          });
        } else {
          resultsDiv.textContent = 'No results found.';
        }
      } catch (error) {
        console.error('Error fetching search results:', error);
        resultsDiv.textContent = 'An error occurred while searching. Please try again.';
      }
    }
  }

## List

In [ ]:
items = [
    {"name": "Teddy Bear", "link": "/holiday_frontend/holiday/toys", "tags": {"all": 1, "teddy": 0, "bear": 0, "toys": 0}},
    {"name": "Lego Set", "link": "/holiday_frontend/holiday/toys", "tags": {"all": 1, "lego": 0, "set": 0, "toys": 0}},
    {"name": "Remote Control Car", "link": "/holiday_frontend/holiday/toys", "tags": {"all": 1, "remote": 0, "control": 0, "car": 0, "toys": 0}},
    {"name": "Holiday Candles", "link": "/holiday_frontend/holiday/home-decor", "tags": {"all": 1, "holiday": 0, "candles": 0, "home-decor": 0}},
    {"name": "Festive Wreath", "link": "/holiday_frontend/holiday/home-decor", "tags": {"all": 1, "festive": 0, "wreath": 0, "home-decor": 0}},
    {"name": "Decorative Ornaments", "link": "/holiday_frontend/holiday/home-decor", "tags": {"all": 1, "decorative": 0, "ornaments": 0, "home-decor": 0}},
    {"name": "Wireless Headphones", "link": "/holiday_frontend/holiday/electronics", "tags": {"all": 1, "wireless": 0, "headphones": 0, "electronics": 0}},
    {"name": "Smartwatch", "link": "/holiday_frontend/holiday/electronics", "tags": {"all": 1, "smartwatch": 0, "electronics": 0}},
    {"name": "Gaming Console", "link": "/holiday_frontend/holiday/electronics", "tags": {"all": 1, "gaming": 0, "console": 0, "electronics": 0}},
    {"name": "Cozy Holiday Sweater", "link": "/holiday_frontend/holiday/clothes", "tags": {"all": 1, "cozy": 0, "holiday": 0, "sweater": 0, "clothes": 0}},
    {"name": "Woolen Scarf", "link": "/holiday_frontend/holiday/clothes", "tags": {"all": 1, "woolen": 0, "scarf": 0, "clothes": 0}},
    {"name": "Winter Gloves", "link": "/holiday_frontend/holiday/clothes", "tags": {"all": 1, "winter": 0, "gloves": 0, "clothes": 0}},
    {"name": "Holiday Cookies", "link": "/holiday_frontend/holiday/food", "tags": {"all": 1, "holiday": 0, "cookies": 0, "food": 0}},
    {"name": "Chocolate Gift Box", "link": "/holiday_frontend/holiday/food", "tags": {"all": 1, "chocolate": 0, "gift": 0, "box": 0, "food": 0}},
    {"name": "Gourmet Cheese Set", "link": "/holiday_frontend/holiday/food", "tags": {"all": 1, "gourmet": 0, "cheese": 0, "set": 0, "food": 0}},
    {"name": "Scented Candle", "link": "/holiday_frontend/holiday/scented", "tags": {"all": 1, "candle": 0, "scented": 0}},
    {"name": "Aromatic Diffuser", "link": "/holiday_frontend/holiday/scented", "tags": {"all": 1, "aromatic": 0, "diffuser": 0, "scented": 0}},
    {"name": "Perfume Gift Set", "link": "/holiday_frontend/holiday/scented", "tags": {"all": 1, "perfume": 0, "gift": 0, "set": 0, "scented": 0}}
]

## List In Use

In [ ]:
@search_api.route("", methods=["GET"])
@token_required()
def search_items():
    query = request.args.get("q", "").lower()
    if not query:
        return jsonify([])

    results = [
        {"name": item["name"], "link": item["link"], "tags": item["tags"]}
        for item in items if query in item["name"].lower()
    ]
    return jsonify(results), 200